In [1]:
import tensorflow as tf
import os
import math
from datetime import datetime
import time
import numpy as np
import segmentation_input
import segmentation_model
slim = tf.contrib.slim

In [2]:
FLAGS = tf.app.flags.FLAGS
NUM_CLASSES = 4

tf.app.flags.DEFINE_string('tfrecord_path', '',
                           """Path to TF records.""")
tf.app.flags.DEFINE_string('checkpoint_dir', '/tmp/segmentation_train/',
                           """Directory where to load checkpoints.""")
tf.app.flags.DEFINE_string('log_dir', '/tmp/segmentation_eval',
                           """Directory where to write event logs """)
tf.app.flags.DEFINE_integer('batch_size', 32,
                            """Batch size.""")
tf.app.flags.DEFINE_integer('eval_interval_secs', 120,
                            """The minimum number of seconds between evaluations.""")
NUM_TEST_SAMPLES = 800
NUM_EVAL=int(math.ceil(float(NUM_TEST_SAMPLES)/float(FLAGS.batch_size)))

['cpu:0', 'cpu:1']
['0', '1']
2


In [5]:
def evaluate():
    with tf.Graph().as_default():
        images, labels = segmentation_input.distorted_inputs(
            [FLAGS.tfrecord_path],
            FLAGS.batch_size,
            NUM_TEST_SAMPLES)

        # Build a Graph that computes the logits predictions from the
        # inference model.
        logits = segmentation_model.inference(images)
        logits_shape = logits.get_shape().as_list()
        label_shape = labels.get_shape().as_list()
        height = label_shape[1]
        width  = label_shape[2]
        num_classes = logits_shape[3]
        predictions = tf.slice(logits,[0,8,8,0],[label_shape[0],height,width,num_classes],name='crop')
        
        _,predicted_labels = tf.nn.top_k(predictions, k=1, sorted=False)
        predicted_labels = tf.squeeze(predicted_labels)
        
        mean_iou,update_op = slim.metrics.streaming_mean_iou(predicted_labels,labels,NUM_CLASSES)
        
        iou = slim.evaluation.evaluation_loop(
            '',
            FLAGS.checkpoint_dir,
            FLAGS.log_dir,
            num_evals=NUM_EVAL,
            eval_op=update_op,
            final_op=mean_iou,
            eval_interval_secs=FLAGS.eval_interval_secs)

In [6]:
def main(unused_args):
    if tf.gfile.Exists(FLAGS.log_dir):
        tf.gfile.DeleteRecursively(FLAGS.log_dir)
    tf.gfile.MakeDirs(FLAGS.log_dir)
    tf.logging.set_verbosity(tf.logging.INFO)
    evaluate()
    
if __name__ == "__main__":
    tf.app.run()

Filling queue with 3198 images before starting to train. This will take a few minutes.


INFO:tensorflow:Waiting for new checkpoint at segmentation_train
INFO:tensorflow:Found new checkpoint at segmentation_train/model.ckpt-3000
INFO:tensorflow:Starting evaluation at 2016-10-13-21:58:59
INFO:tensorflow:Executing eval ops
INFO:tensorflow:Executing eval_op 1/10
INFO:tensorflow:Executing eval_op 2/10
INFO:tensorflow:Executing eval_op 3/10
INFO:tensorflow:Executing eval_op 4/10
INFO:tensorflow:Executing eval_op 5/10
INFO:tensorflow:Executing eval_op 6/10
INFO:tensorflow:Executing eval_op 7/10
INFO:tensorflow:Executing eval_op 8/10
INFO:tensorflow:Executing eval_op 9/10
INFO:tensorflow:Executing eval_op 10/10
INFO:tensorflow:Executing final op
INFO:tensorflow:Executing summary op
INFO:tensorflow:metric has value 0.530059
INFO:tensorflow:test
INFO:tensorflow:Finished evaluation at 2016-10-13-21:59:04


KeyboardInterrupt: 